In [1]:
import torch
import torch.nn as nn
from torchsummary import summary
from torchviz import make_dot

## device
#device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class Inception(nn.Module):
    def __init__(self,input_ch,out1x1,red3x3,out3x3,red5x5,out5x5,pool_proj):
        super(Inception,self).__init__()
        self.arm1x1=nn.Sequential(
            nn.Conv2d(input_ch,out1x1,kernel_size=1),
            nn.BatchNorm2d(out1x1),
            nn.ReLU(True),
        )

        self.arm3x3=nn.Sequential(
            nn.Conv2d(input_ch,red3x3,kernel_size=1),
            nn.BatchNorm2d(red3x3),
            nn.ReLU(True),
            nn.Conv2d(red3x3,out3x3,kernel_size=3,padding=1),
            nn.BatchNorm2d(out3x3),
            nn.ReLU(True),
        )
       
        self.arm5x5=nn.Sequential(
            nn.Conv2d(input_ch,red5x5,kernel_size=1),
            nn.BatchNorm2d(red5x5),
            nn.ReLU(True),
            nn.Conv2d(red5x5,out5x5,5,padding=2),
            nn.BatchNorm2d(out5x5),
            nn.ReLU(True),
        )       
 
        self.arm_pool=nn.Sequential(
            nn.MaxPool2d(kernel_size=3,stride=1,padding=1),
            nn.Conv2d(input_ch,pool_proj,kernel_size=1),
            nn.BatchNorm2d(pool_proj),
            nn.ReLU(True)
        )
    def forward(self,m):
        arm1=self.arm1x1(m)
        arm3=self.arm3x3(m)
        arm5=self.arm5x5(m)
        pool_proj=self.arm_pool(m)
        return torch.cat([arm1,arm3,arm5,pool_proj],1)

class GoogleNet(nn.Module):
    def __init__(self,n_class):
        super(GoogleNet,self).__init__()
        self.conv_block=nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=64,kernel_size=7,stride=2,padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=3,stride=2,padding=1),
            nn.Conv2d(in_channels=64,out_channels=192,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(192),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=3,stride=2,padding=1),
        )
        self.inception3a=Inception(192,64,96,128,16,32,32)
        self.inception3b=Inception(256,128,128,192,32,96,64)
        self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        self.inception4a=Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b=Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c=Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d=Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e=Inception(528, 256, 160, 320, 32, 128, 128)
        self.inception5a=Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b=Inception(832, 384, 192, 384, 48, 128, 128)

        self.avgpool=nn.AvgPool2d(kernel_size=7,stride=1)
        self.dropout=nn.Dropout(0.4)
        self.fc=nn.Linear(1024,n_class)

    def forward(self,m):
        m=self.conv_block(m)
        m=self.inception3a(m)
        m=self.inception3b(m)
        m=self.maxpool(m)
        m=self.inception4a(m)
        m=self.inception4b(m)
        m=self.inception4c(m)
        m=self.inception4d(m)
        m=self.inception4e(m)
        m=self.maxpool(m)
        m=self.inception5a(m)
        m=self.inception5b(m)
        m=self.avgpool(m)
        m=m.view(m.size(0),-1)
        m=self.dropout(m)
        m=self.fc(m)
        return m

c:\Users\Abbos\.conda\envs\lanedet\lib\site-packages\tqdm-4.64.0-py3.8.egg\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
import torch
import torchvision
from torchvision import datasets,transforms
import matplotlib.pyplot as plt
import numpy as np
device= device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# the location of image data(main dir)
dataset='dataset'  
# Custom dataset class
class WeatherDataset(torch.utils.data.Dataset):
    def __init__(self,dataset,transform=None):
        super(WeatherDataset,self).__init__()
        self.dataset=dataset
        self.transform=transform

        self.images=[]
        self.labels=[]

        for class_name in os.listdir(dataset):
            class_dir=os.path.join(dataset,class_name)
            if os.path.isdir(class_dir):
                if class_name=='cloudy':
                 label=0
            if class_name=='sunrise':
                label=1
            if class_name=='rainy':
                 label=2
            if class_name=='shine':
                label=3  
            for image in os.listdir(class_dir):
                image_path=os.path.join(class_dir,image)
                self.images.append(image_path)
                self.labels.append(label)    
    def __len__(self):
        return len(self.images)
    def __getitem__(self,idx):
        image=torchvision.datasets.folder.pil_loader(self.images[idx])
        if self.transform:
           image=self.transform(image)
        return image, self.labels[idx] 
transform=transforms.Compose([
   transforms.RandomRotation(10),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])
data_set=WeatherDataset(dataset,transform=transform)
trainset,validset=torch.utils.data.random_split(data_set,[1000,125])
train_loader=torch.utils.data.DataLoader(trainset,batch_size=16,shuffle=True)
valid_loader=torch.utils.data.DataLoader(validset,batch_size=16,shuffle=True)

In [ ]:
#data_set=WeatherDataset(dataset,transform=transform)
#print(data_set.__len__())
#print(data_set.__getitem__(4))
samples,labels=iter(train_loader).next()
classes={0:'cloudy',1:'sunrise',2:'rainy',3:'shine'}
fig=plt.figure(figsize=(8,8))
for i in range(16):
    a=fig.add_subplot(2,2,i+1)
    a.set_title(classes[labels[i].item()])
    a.axis('off')
    a.imshow(np.transpose(samples[i].numpy(),(1,2,0)))
plt.subplots_adjust(bottom=0.2, top=0.6, hspace=0)    


In [9]:
import torch.nn as nn
import torch.optim as optim
import time
from nn import *

device= device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=GoogleNet(n_class=4).to(device)
criterion=nn.CrossEntropyLoss()
optimzer=optim.SGD(model.parameters(),lr=0.01,momentum=0.9)
train_loss,val_loss,train_cor=0,0,0
total,total_val=0,0
train_acc=[]
train_losses=[]
epochs=100
for epoch in range(epochs):
   train_loss=0.0
   start=time.time()
   for i,data in enumerate(train_loader,0):
        inputs,labels=data
        inputs,labels=inputs.to(device),labels.to(device)
        optimzer.zero_grad()
        outputs=model(inputs)
        loss=criterion(outputs,labels)
        loss.backward()
        optimzer.step()
        train_loss+=loss.item()
        _,y_pred=torch.max(outputs,1)    
        y_pred[y_pred>=0.5]=1
        y_pred[y_pred<=0.5]=0
        train_cor+=(y_pred==labels).sum().item()
        total+=labels.size(0)
        train_losses=train_loss/len(train_loader)
        train_acc=100*train_cor/total
        ## validation checking
        val_loss=0
        val_acc=0
        valid_correct = 0
        '''
        for valid_data in valid_loader:
            with torch.no_grad():
                model.eval()
                val_x,val_y=valid_data
                val_x, val_y = val_x.to(device), val_y.to(device)
                pred = model(val_x)
                loss = criterion(pred, val_y)
                val_loss += loss.item()
            
                _,y_pred = torch.max(pred,1)
                y_pred[y_pred >= 0.5] = 1
                y_pred[y_pred < 0.5] = 0
                valid_correct +=(y_pred==val_y).sum().item()
                total_val+=val_y.size(0)
                valid_losses=val_loss/len(valid_loader)   
        #train_acc = train_correct/len(train_loader.dataset)
                valid_acc = 100*valid_correct/total_val
        '''
   print(f'{time.time()-start:.3f}sec : [Epoch {epoch+1}/{epochs}] train loss: {train_loss:.3f} -> training accuracy: {train_acc:.4f}') # -> validation loss: {val_loss:.3f} -> validation accuracy: {valid_acc:.3f}')

print('Finished training')    

17.772sec : [Epoch 1/100] train loss: 53.326 -> training accuracy: 43.3000
18.199sec : [Epoch 2/100] train loss: 40.597 -> training accuracy: 45.7000
17.487sec : [Epoch 3/100] train loss: 35.850 -> training accuracy: 47.3333
17.300sec : [Epoch 4/100] train loss: 29.491 -> training accuracy: 48.4750
17.298sec : [Epoch 5/100] train loss: 31.985 -> training accuracy: 49.1800
17.326sec : [Epoch 6/100] train loss: 22.729 -> training accuracy: 49.9333
17.472sec : [Epoch 7/100] train loss: 21.020 -> training accuracy: 50.4429
17.666sec : [Epoch 8/100] train loss: 22.116 -> training accuracy: 50.8125
17.349sec : [Epoch 9/100] train loss: 18.251 -> training accuracy: 51.2111
18.013sec : [Epoch 10/100] train loss: 16.315 -> training accuracy: 51.5400
17.325sec : [Epoch 11/100] train loss: 16.298 -> training accuracy: 51.7818
17.201sec : [Epoch 12/100] train loss: 19.099 -> training accuracy: 51.9917
17.262sec : [Epoch 13/100] train loss: 17.452 -> training accuracy: 52.1308
17.264sec : [Epoch 14